# Azure AI Agent Service এবং Semantic Kernel ব্যবহার করে RAG

এই কোড স্নিপেটটি দেখায় কীভাবে `Azure AI Agent Service` এবং `Semantic Kernel` ব্যবহার করে রিট্রিভাল-অগমেন্টেড জেনারেশন (RAG) এর জন্য একটি Azure AI এজেন্ট তৈরি এবং পরিচালনা করা যায়। এজেন্টটি পুনরুদ্ধার করা প্রসঙ্গের ভিত্তিতে ব্যবহারকারীর প্রশ্ন প্রক্রিয়াকরণ করে এবং সেই অনুযায়ী সঠিক উত্তর প্রদান করে।


SQLite সংস্করণ সংশোধন  
যদি আপনি এই ত্রুটির সম্মুখীন হন:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

আপনার নোটবুকের শুরুতে এই কোড ব্লকটি আনকমেন্ট করুন:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### প্যাকেজ আমদানি করা  
নিচের কোডটি প্রয়োজনীয় প্যাকেজগুলো আমদানি করে:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# সেমান্টিক কোর ও অ্যাজুর এআই এজেন্ট সার্ভিসের সাথে রিট্রিভাল-অগমেন্টেড জেনারেশন

এই উদাহরণটি দেখায় কীভাবে **অ্যাজুর এআই এজেন্ট সার্ভিস** ব্যবহার করে একটি ভাষা মডেলকে আপলোড করা ডকুমেন্ট থেকে প্রাসঙ্গিক তথ্যের সাথে সংযুক্ত করে **রিট্রিভাল-অগমেন্টেড জেনারেশন (RAG)** সম্পাদন করা যায়।

### এটি কীভাবে কাজ করে

1. **ডকুমেন্ট আপলোড**: একটি মার্কডাউন ফাইল (document.md) যার মধ্যে তথ্য (কন্টোসোর ভ্রমণ বিমা নীতি) রয়েছে, এজেন্ট সার্ভিসে আপলোড করা হয়।

2. **ভেক্টর স্টোর তৈরি**: ডকুমেন্টটি একটি ভেক্টর স্টোরে সূচীকৃত করা হয়, যাতে এর বিষয়বস্তুর উপর সেমান্টিক সার্চ করা যায়।

3. **এজেন্ট কনফিগারেশন**: `gpt-4o` মডেল ব্যবহার করে একটি এজেন্ট নিম্নলিখিত কঠোর নির্দেশাবলী সহ কনফিগার করা হয়:
   - শুধুমাত্র ডকুমেন্ট থেকে প্রাপ্ত বিষয়বস্তুর উপর ভিত্তি করে প্রশ্নের উত্তর দিতে হবে।
   - যদি প্রশ্নটি প্রাসঙ্গিক না হয়, তবে উত্তর দিতে অস্বীকার করতে হবে।

4. **ফাইল সার্চ টুল ইন্টিগ্রেশন**: এজেন্টের সাথে `FileSearchTool` নিবন্ধিত করা হয়, যা মডেলকে ইনফারেন্সের সময় সূচীকৃত ডকুমেন্ট থেকে প্রাসঙ্গিক অংশ খুঁজে বের করতে সক্ষম করে।

5. **ব্যবহারকারীর সাথে যোগাযোগ**: ব্যবহারকারীরা প্রশ্ন করতে পারেন। যদি ডকুমেন্টে প্রাসঙ্গিক তথ্য পাওয়া যায়, এজেন্ট একটি ভিত্তিপ্রাপ্ত উত্তর তৈরি করে।  
   যদি না পাওয়া যায়, এজেন্ট স্পষ্টভাবে জানায় যে ডকুমেন্টে পর্যাপ্ত তথ্য নেই।


### প্রধান ফাংশন


সঠিক প্রমাণীকরণ প্রসঙ্গ নিশ্চিত করার জন্য Azure CLI ব্যবহার করে প্রথমে `az login` চালান, যখন `DefaultAzureCredential` ব্যবহার করবেন। Azure AI Agent Service API কী ব্যবহার করে না।


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

আপনার অনুরোধটি অনুধাবন করেছি। অনুগ্রহ করে আমাকে সেই Markdown ফাইলটি প্রদান করুন, যাতে আমি এটি বাংলায় অনুবাদ করতে পারি।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় রচিত সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হচ্ছে। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
